In [1]:
import pyspark
from importlib import reload

from pyspark.sql import SparkSession
import os
import pandas as pd
import sys
sys.path.insert(0, '/home/jovyan/work')
from script.helper.db_conn import db_connection
from datetime import datetime
from datetime import timezone

from script.helper.conn_prop import connection_properties
from script.helper.init_spark import initiate_spark
from script.etl.extract import *
from script.etl.load import *
from script.etl.transform import Transform
import pandas as pd
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import to_date
from pyspark.sql.functions import *

In [1]:

people=extract_csv('../data/people.csv')

NameError: name 'extract_csv' is not defined

In [3]:
relationships=extract_csv('../data/relationship.csv')

===== Start Extracting ../data/relationship.csv data =====
====== Failed to Extract Data ../data/relationship.csv ======,
 [PATH_NOT_FOUND] Path does not exist: file:/home/jovyan/work/data/relationship.csv.


In [4]:
relationships=extract_csv('../data/relationships.csv')

===== Start Extracting ../data/relationships.csv data =====
===== Success Extracting ../data/relationships.csv data =====


In [5]:
table_names=["acquisition","company","funds","investments","ipos","funding_rounds"]
for name in table_names:
    globals()[f'{name}']=extract_db_src(name)

===== Start Extracting acquisition data =====
===== Success Extracting acquisition data =====
===== Start Extracting company data =====
===== Success Extracting company data =====
===== Start Extracting funds data =====
===== Success Extracting funds data =====
===== Start Extracting investments data =====
===== Success Extracting investments data =====
===== Start Extracting ipos data =====
===== Success Extracting ipos data =====
===== Start Extracting funding_rounds data =====
===== Success Extracting funding_rounds data =====


In [6]:
relationships.show()

+---------------+----------------+----------------------+--------------------+--------------------+-------+--------+--------------------+--------------------+--------------------+
|relationship_id|person_object_id|relationship_object_id|            start_at|              end_at|is_past|sequence|               title|          created_at|          updated_at|
+---------------+----------------+----------------------+--------------------+--------------------+-------+--------+--------------------+--------------------+--------------------+
|              1|             p:2|                   c:1|                NULL|                NULL|  false|       8|Co-Founder/CEO/Bo...|2007-05-25 07:03:...|2013-06-03 09:58:...|
|              2|             p:3|                   c:1|                NULL|                NULL|  false|  279242|        VP Marketing|2007-05-25 07:04:...|2010-05-21 16:31:...|
|              3|             p:4|                   c:3|                NULL|                NULL| 

In [7]:
people.show()

+---------+---------+----------+----------+--------------------+--------------------+
|people_id|object_id|first_name| last_name|          birthplace|    affiliation_name|
+---------+---------+----------+----------+--------------------+--------------------+
|        1|      p:2|       Ben|   Elowitz|                NULL|           Blue Nile|
|        2|      p:3|     Kevin|  Flaherty|                NULL|            Wetpaint|
|        3|      p:4|      Raju|   Vegesna|                NULL|                Zoho|
|        4|      p:5|       Ian|     Wenig|                NULL|                Zoho|
|        5|      p:6|     Kevin|      Rose|         Redding, CA|        i/o Ventures|
|        6|      p:7|       Jay|   Adelson|         Detroit, MI|                Digg|
|        7|      p:8|      Owen|     Byrne|                NULL|                Digg|
|        8|      p:9|       Ron|Gorodetzky|                NULL|                Digg|
|        9|     p:10|      Mark|Zuckerberg|           

In [8]:
step="Data Staging"
process="Load"



In [9]:
load_stg(people,"people")

===== Get only new data =====
===== Start Extracting people data =====
===== Success Extracting people data =====
===== Already got only New Data  =====
===== Start Loading people new data =====
===== Success Loading people new data =====


In [10]:
people.dtypes

[('people_id', 'string'),
 ('object_id', 'string'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('birthplace', 'string'),
 ('affiliation_name', 'string')]

In [11]:
relationships.dtypes

[('relationship_id', 'string'),
 ('person_object_id', 'string'),
 ('relationship_object_id', 'string'),
 ('start_at', 'string'),
 ('end_at', 'string'),
 ('is_past', 'string'),
 ('sequence', 'string'),
 ('title', 'string'),
 ('created_at', 'string'),
 ('updated_at', 'string')]

In [12]:


relationships=relationships.withColumn("start_at",to_date( relationships['start_at'], "yyyy-mm-dd")) 
relationships=relationships.withColumn("end_at",to_date( relationships['end_at'], "yyyy-mm-dd")) 
relationships=relationships.withColumn("created_at",to_timestamp( relationships['created_at'], "yyyy-mm-dd HH:mm:ss")) 
relationships=relationships.withColumn("updated_at",to_timestamp( relationships['updated_at'], "yyyy-mm-dd HH:mm:ss")) 

In [13]:
relationships.show()

+---------------+----------------+----------------------+----------+----------+-------+--------+--------------------+-------------------+-------------------+
|relationship_id|person_object_id|relationship_object_id|  start_at|    end_at|is_past|sequence|               title|         created_at|         updated_at|
+---------------+----------------+----------------------+----------+----------+-------+--------+--------------------+-------------------+-------------------+
|              1|             p:2|                   c:1|      NULL|      NULL|  false|       8|Co-Founder/CEO/Bo...|2007-01-25 07:03:54|2013-01-03 09:58:46|
|              2|             p:3|                   c:1|      NULL|      NULL|  false|  279242|        VP Marketing|2007-01-25 07:04:16|2010-01-21 16:31:34|
|              3|             p:4|                   c:3|      NULL|      NULL|  false|       4|          Evangelist|2007-01-25 19:33:03|2013-01-29 13:36:58|
|              4|             p:5|                  

In [14]:
relationships.dtypes

[('relationship_id', 'string'),
 ('person_object_id', 'string'),
 ('relationship_object_id', 'string'),
 ('start_at', 'date'),
 ('end_at', 'date'),
 ('is_past', 'string'),
 ('sequence', 'string'),
 ('title', 'string'),
 ('created_at', 'timestamp'),
 ('updated_at', 'timestamp')]

In [15]:
load_stg(relationships,"relationships")

===== Get only new data =====
===== Start Extracting relationships data =====
===== Success Extracting relationships data =====
===== Already got only New Data  =====
===== Start Loading relationships new data =====
===== Success Loading relationships new data =====


In [16]:
table_names=["acquisition","company","funds","investments","ipos","funding_rounds"]
for name in table_names:
    load_stg(globals()[f'{name}'],name)

===== Get only new data =====
===== Start Extracting acquisition data =====
===== Success Extracting acquisition data =====
===== Already got only New Data  =====
===== Start Loading acquisition new data =====
===== Success Loading acquisition new data =====
===== Get only new data =====
===== Start Extracting company data =====
===== Success Extracting company data =====
===== Already got only New Data  =====
===== Start Loading company new data =====
===== Success Loading company new data =====
===== Get only new data =====
===== Start Extracting funds data =====
===== Success Extracting funds data =====
===== Already got only New Data  =====
===== Start Loading funds new data =====
===== Success Loading funds new data =====
===== Get only new data =====
===== Start Extracting investments data =====
===== Success Extracting investments data =====
===== Already got only New Data  =====
===== Start Loading investments new data =====
===== Success Loading investments new data =====
====

# DATA STAGING ETL

## EXTRACT

In [17]:
table_names=["people","relationships","acquisition","company","funds","investments","ipos","funding_rounds"]
for name in table_names:
    globals()[f'stg_{name}']=extract_db_stg(name)

===== Start Extracting people data =====
===== Success Extracting people data =====
===== Start Extracting relationships data =====
===== Success Extracting relationships data =====
===== Start Extracting acquisition data =====
===== Success Extracting acquisition data =====
===== Start Extracting company data =====
===== Success Extracting company data =====
===== Start Extracting funds data =====
===== Success Extracting funds data =====
===== Start Extracting investments data =====
===== Success Extracting investments data =====
===== Start Extracting ipos data =====
===== Success Extracting ipos data =====
===== Start Extracting funding_rounds data =====
===== Success Extracting funding_rounds data =====


In [18]:
dim_term_code=stg_acquisition.select(stg_acquisition['term_code']).distinct()


In [19]:
load_dwh(dim_term_code,"dim_term_code")

===== Get only new data =====
===== Start Extracting dim_term_code data =====
===== Success Extracting dim_term_code data =====
===== Already got only New Data  =====
===== Start Loading dim_term_code new data =====
===== Success Loading dim_term_code new data =====


In [20]:
stock_symbol=stg_ipos.select(stg_ipos['stock_symbol']).distinct()


In [21]:
load_dwh(stock_symbol,"dim_stock_symbol")

===== Get only new data =====
===== Start Extracting dim_stock_symbol data =====
===== Success Extracting dim_stock_symbol data =====
===== Already got only New Data  =====
===== Start Loading dim_stock_symbol new data =====
===== Success Loading dim_stock_symbol new data =====


In [22]:
RENAME_COLS = {
                "office_id": "company_nk_id"}
dim_company = stg_company.withColumnsRenamed(colsMap = RENAME_COLS)

In [23]:
columns_to_drop = ['created_at', 'updated_at']
dim_company = dim_company.drop(*columns_to_drop)


In [24]:
load_dwh(dim_company,"dim_company")

===== Get only new data =====
===== Start Extracting dim_company data =====
===== Success Extracting dim_company data =====
===== Already got only New Data  =====
===== Start Loading dim_company new data =====
===== Success Loading dim_company new data =====


In [25]:
RENAME_COLS = {
                "people_id": "people_nk_id"}
dim_people = stg_people.withColumnsRenamed(colsMap = RENAME_COLS)
columns_to_drop = ['created_at', 'updated_at']
dim_people = dim_people.drop(*columns_to_drop)

In [26]:
dim_people=dim_people.withColumn("people_nk_id",dim_people ["people_nk_id"].cast("int"))

In [27]:
dim_people.dtypes

[('people_nk_id', 'int'),
 ('object_id', 'string'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('birthplace', 'string'),
 ('affiliation_name', 'string')]

In [28]:
load_dwh(dim_people,"dim_people")

===== Get only new data =====
===== Start Extracting dim_people data =====
===== Success Extracting dim_people data =====
===== Already got only New Data  =====
===== Start Loading dim_people new data =====
===== Success Loading dim_people new data =====


In [29]:
dim_people.show()

+------------+---------+----------+---------+----------+----------------+
|people_nk_id|object_id|first_name|last_name|birthplace|affiliation_name|
+------------+---------+----------+---------+----------+----------------+
|         224|    p:248|       Jim|    Young|      NULL|      HOT or NOT|
|         364|    p:414|     Brian|   Mulloy|      NULL|          Swivel|
|         505|    p:569|    Renaud|   Visage|      NULL|      Eventbrite|
|         684|    p:779|      Eric| Gonzales|      NULL|           Ujogo|
|         981|   p:1127|     Alain| Rossmann|      NULL|            Vudu|
|        1182|   p:1361|    Debbie| Dembecki|      NULL|       LiveWorld|
|        1657|   p:1914|      Igor| Jablokov|      NULL|             IBM|
|        1863|   p:2164|     Shawn|     Colo|      NULL|    Demand Media|
|        2357|   p:2775|     Henry|     Wong|      NULL|         Zipidee|
|        2649|   p:3189|     Kevin|  Fortuna|      NULL|           Quigo|
|        2723|   p:3294|     Lukas|  Z

In [30]:
dim_time=extract_db_dwh("dim_time",step="Transform",process="Extract DWH for joining")

===== Start Extracting dim_time data =====
===== Success Extracting dim_time data =====


In [31]:
dim_people=extract_db_dwh("dim_people",step="Transform",process="Extract DWH for joining")

===== Start Extracting dim_people data =====
===== Success Extracting dim_people data =====


In [32]:
dim_company=extract_db_dwh("dim_company",step="Transform",process="Extract DWH for joining")

===== Start Extracting dim_company data =====
===== Success Extracting dim_company data =====


In [33]:
dim_date=extract_db_dwh("dim_date",step="Transform",process="Extract DWH for joining")

===== Start Extracting dim_date data =====
===== Success Extracting dim_date data =====


In [34]:
dim_term_code=extract_db_dwh("dim_term_code",step="Transform",process="Extract DWH for joining")

===== Start Extracting dim_term_code data =====
===== Success Extracting dim_term_code data =====


In [35]:
dim_stock_symbol=extract_db_dwh("dim_stock_symbol",step="Transform",process="Extract DWH for joining")

===== Start Extracting dim_stock_symbol data =====
===== Success Extracting dim_stock_symbol data =====


In [36]:
dim_date=dim_date.select(['date_id','date_actual'])
dim_company=dim_company.select(['company_id','object_id'])
dim_people=dim_people.select(['people_id','object_id'])

In [37]:
relationships=stg_relationships.join(dim_date, stg_relationships.start_at == dim_date.date_actual, 'left')

In [38]:
columns_to_drop = ['created_at', 'updated_at','date_actual','start_at']
relationships = relationships.drop(*columns_to_drop)
RENAME_COLS = {
                "date_id": "start_at"}
relationships = relationships.withColumnsRenamed(colsMap = RENAME_COLS)


In [39]:
relationships=relationships.join(dim_date, relationships.end_at == dim_date.date_actual, 'left')

In [40]:
columns_to_drop = ['date_actual','end_at']
relationships = relationships.drop(*columns_to_drop)
RENAME_COLS = {
                "date_id": "end_at"}
relationships = relationships.withColumnsRenamed(colsMap = RENAME_COLS)

In [41]:
relationships=relationships.join(dim_company, relationships.relationship_object_id == dim_company.object_id, 'left')


In [42]:
columns_to_drop = ['relationship_object_id','object_id']
relationships = relationships.drop(*columns_to_drop)
RENAME_COLS = {
                "company_id": "relationship_object_id"}
relationships = relationships.withColumnsRenamed(colsMap = RENAME_COLS)

In [43]:
relationships=relationships.join(dim_people, relationships.person_object_id == dim_people.object_id, 'left')


In [44]:
columns_to_drop = ['object_id','person_object_id']
relationships = relationships.drop(*columns_to_drop)

In [45]:
RENAME_COLS = {
                "relationship_id": "relationship_nk_id"}
relationships = relationships.withColumnsRenamed(colsMap = RENAME_COLS)

In [46]:
relationships=relationships.withColumn("relationship_nk_id",relationships ["relationship_nk_id"].cast("int"))

In [47]:
relationships=relationships.withColumn("is_past",when(relationships.is_past == "true", True)
                                   .when(relationships.is_past == "false", False))

In [48]:
load_dwh(relationships,"fct_person_relationship")

===== Get only new data =====
===== Start Extracting fct_person_relationship data =====
===== Success Extracting fct_person_relationship data =====
===== Already got only New Data  =====
===== Start Loading fct_person_relationship new data =====
===== Success Loading fct_person_relationship new data =====


In [49]:
funding_rounds=stg_funding_rounds.join(dim_date, stg_funding_rounds.funded_at == dim_date.date_actual, 'left')

In [50]:
columns_to_drop = ['created_at', 'updated_at','date_actual','funded_at']
funding_rounds = funding_rounds.drop(*columns_to_drop)
RENAME_COLS = {
                "date_id": "funded_at"}
funding_rounds = funding_rounds.withColumnsRenamed(colsMap = RENAME_COLS)


In [51]:
funding_rounds=funding_rounds.join(dim_company, funding_rounds.object_id == dim_company.object_id, 'left')


In [52]:
columns_to_drop = ['object_id','object_id']
funding_rounds = funding_rounds.drop(*columns_to_drop)
RENAME_COLS = {
                "company_id": "object_id","funding_round_id":"funding_round_nk_id" }
funding_rounds = funding_rounds.withColumnsRenamed(colsMap = RENAME_COLS)

In [53]:
funding_rounds.dtypes

[('funding_round_nk_id', 'int'),
 ('funding_round_type', 'string'),
 ('funding_round_code', 'string'),
 ('raised_amount_usd', 'decimal(15,2)'),
 ('raised_amount', 'decimal(15,2)'),
 ('raised_currency_code', 'string'),
 ('pre_money_valuation_usd', 'decimal(15,2)'),
 ('pre_money_valuation', 'decimal(15,2)'),
 ('pre_money_currency_code', 'string'),
 ('post_money_valuation_usd', 'decimal(15,2)'),
 ('post_money_valuation', 'decimal(15,2)'),
 ('post_money_currency_code', 'string'),
 ('participants', 'string'),
 ('is_first_round', 'boolean'),
 ('is_last_round', 'boolean'),
 ('source_url', 'string'),
 ('source_description', 'string'),
 ('created_by', 'string'),
 ('funded_at', 'int'),
 ('object_id', 'string')]

In [54]:
funding_rounds=funding_rounds.withColumn("is_first_round",when(funding_rounds.is_first_round == "true", True)
                                   .when(funding_rounds.is_first_round == "false", False))

In [55]:
load_dwh(funding_rounds,"fct_funding_rounds")

===== Get only new data =====
===== Start Extracting fct_funding_rounds data =====
===== Success Extracting fct_funding_rounds data =====
===== Already got only New Data  =====
===== Start Loading fct_funding_rounds new data =====
===== Success Loading fct_funding_rounds new data =====


In [56]:
funds=stg_funds.join(dim_date, stg_funds.funded_at == dim_date.date_actual, 'left')

In [57]:
columns_to_drop = ['created_at', 'updated_at','date_actual','funded_at']
funds = funds.drop(*columns_to_drop)
RENAME_COLS = {
                "date_id": "funded_at", "fund_id":"fund_nk_id" }
funds = funds.withColumnsRenamed(colsMap = RENAME_COLS)

In [58]:
funds.dtypes

[('fund_nk_id', 'string'),
 ('object_id', 'string'),
 ('name', 'string'),
 ('raised_amount', 'decimal(15,2)'),
 ('raised_currency_code', 'string'),
 ('source_url', 'string'),
 ('source_description', 'string'),
 ('funded_at', 'int')]

In [59]:
funds=funds.join(dim_company, funds.object_id == dim_company.object_id, 'left')
columns_to_drop = ['object_id','object_id']
funds = funds.drop(*columns_to_drop)
RENAME_COLS = {
                "company_id": "object_id" }
funds = funds.withColumnsRenamed(colsMap = RENAME_COLS)


In [60]:
load_dwh(funds,"fct_funds")

===== Get only new data =====
===== Start Extracting fct_funds data =====
===== Success Extracting fct_funds data =====
===== Already got only New Data  =====
===== Start Loading fct_funds new data =====
===== Success Loading fct_funds new data =====


In [61]:
acquisition=stg_acquisition.join(dim_company, stg_acquisition.acquiring_object_id == dim_company.object_id, 'left')

In [62]:
columns_to_drop = ['acquiring_object_id','object_id','created_at','updated_at']
acquisition = acquisition.drop(*columns_to_drop)
RENAME_COLS = {
                "company_id": "acquiring_object_id","acquisition_id":"acquisition_nk_id" }
acquisition = acquisition.withColumnsRenamed(colsMap = RENAME_COLS)

In [63]:
acquisition.show()


+-----------------+------------------+--------------+------------+-------------------+-------------------+--------------------+--------------------+--------------------+
|acquisition_nk_id|acquired_object_id|     term_code|price_amount|price_currency_code|        acquired_at|          source_url|  source_description| acquiring_object_id|
+-----------------+------------------+--------------+------------+-------------------+-------------------+--------------------+--------------------+--------------------+
|              148|            c:2108|          cash|        0.00|                USD|2007-08-01 00:00:00|http://www.dating...|Ashley Madison ac...|fadd67fa-73df-448...|
|             1580|           c:26918|              |  7350000.00|                USD|2008-06-09 00:00:00|http://findarticl...|INX Acquires Acce...|a59dea0e-5206-478...|
|             1591|           c:26939|          cash| 18000000.00|                USD|2008-04-16 00:00:00|http://zes.zebra....|Zebra Technologie...|ff

In [64]:
acquisition=acquisition.join(dim_company, acquisition.acquired_object_id == dim_company.object_id, 'left')
columns_to_drop = ['acquired_object_id','object_id']
acquisition = acquisition.drop(*columns_to_drop)
RENAME_COLS = {
                "company_id": "acquired_object_id" }
acquisition = acquisition.withColumnsRenamed(colsMap = RENAME_COLS)

In [65]:
acquisition=acquisition.join(dim_term_code, acquisition.term_code == dim_term_code.term_code, 'left')


In [66]:
columns_to_drop = ['term_code','term_code']
acquisition = acquisition.drop(*columns_to_drop)


In [67]:
acquisition=acquisition.withColumn('acquired_at', to_date( acquisition['acquired_at'], "yyyy-MM-dd"))

In [68]:
acquisition=acquisition.join(dim_date, acquisition.acquired_at == dim_date.date_actual, 'left')
columns_to_drop = ['date_actual','acquired_at']
acquisition = acquisition.drop(*columns_to_drop)
RENAME_COLS = {
                "date_id": "acquired_at"}
acquisition = acquisition.withColumnsRenamed(colsMap = RENAME_COLS)

In [69]:
load_dwh(acquisition,"fct_acquisition")

===== Get only new data =====
===== Start Extracting fct_acquisition data =====
===== Success Extracting fct_acquisition data =====
===== Already got only New Data  =====
===== Start Loading fct_acquisition new data =====
===== Success Loading fct_acquisition new data =====


In [70]:
acquisition.dtypes

[('acquisition_nk_id', 'int'),
 ('price_amount', 'decimal(15,2)'),
 ('price_currency_code', 'string'),
 ('source_url', 'string'),
 ('source_description', 'string'),
 ('acquiring_object_id', 'string'),
 ('acquired_object_id', 'string'),
 ('term_code_id', 'string'),
 ('acquired_at', 'int')]

In [71]:
ipos=stg_ipos.join(dim_company, stg_ipos.object_id == dim_company.object_id, 'left')
columns_to_drop = ['object_id','object_id','created_at','updated_at']
ipos = ipos.drop(*columns_to_drop)
RENAME_COLS = {
                "company_id": "object_id", "ipo_id": "ipo_nk_id" }
ipos = ipos.withColumnsRenamed(colsMap = RENAME_COLS)

In [72]:
ipos.dtypes

[('ipo_nk_id', 'string'),
 ('valuation_amount', 'decimal(15,2)'),
 ('valuation_currency_code', 'string'),
 ('raised_amount', 'decimal(15,2)'),
 ('raised_currency_code', 'string'),
 ('public_at', 'timestamp'),
 ('stock_symbol', 'string'),
 ('source_url', 'string'),
 ('source_description', 'string'),
 ('object_id', 'string')]

In [73]:
ipos=ipos.withColumn('public_at', to_date( ipos['public_at'], "yyyy-MM-dd"))
ipos=ipos.join(dim_date, ipos.public_at == dim_date.date_actual, 'left')
columns_to_drop = ['date_actual','public_at']
ipos = ipos.drop(*columns_to_drop)
RENAME_COLS = {
                "date_id": "public_at"}
ipos = ipos.withColumnsRenamed(colsMap = RENAME_COLS)

In [74]:
ipos=ipos.join(dim_stock_symbol, ipos.stock_symbol == dim_stock_symbol.stock_symbol, 'left')
columns_to_drop = ['stock_symbol','stock_symbol']
ipos = ipos.drop(*columns_to_drop)
RENAME_COLS = {
                "stock_symbol_id": "stock_symbol"}
ipos = ipos.withColumnsRenamed(colsMap = RENAME_COLS)

In [75]:
load_dwh(ipos,"fct_ipos")

===== Get only new data =====
===== Start Extracting fct_ipos data =====
===== Success Extracting fct_ipos data =====
===== Already got only New Data  =====
===== Start Loading fct_ipos new data =====
===== Success Loading fct_ipos new data =====


In [76]:
fct_funding_rounds=extract_db_dwh("fct_funding_rounds",step="Transform",process="Extract DWH for joining")
fct_funding_rounds=fct_funding_rounds.select(['funding_round_id','funding_round_nk_id'])
RENAME_COLS = {
                "funding_round_id": "id"}
fct_funding_rounds = fct_funding_rounds.withColumnsRenamed(colsMap = RENAME_COLS)


===== Start Extracting fct_funding_rounds data =====
===== Success Extracting fct_funding_rounds data =====


In [77]:
investments=stg_investments.join(dim_company, stg_investments.funded_object_id == dim_company.object_id, 'left')
columns_to_drop = ['funded_object_id','object_id','created_at','updated_at']
investments = investments.drop(*columns_to_drop)
RENAME_COLS = {
                "company_id": "funded_object_id", "investment_id": "investment_nk_id" }
investments = investments.withColumnsRenamed(colsMap = RENAME_COLS)

In [78]:
investments.dtypes

[('investment_nk_id', 'int'),
 ('funding_round_id', 'int'),
 ('investor_object_id', 'string'),
 ('funded_object_id', 'string')]

In [79]:
investments=investments.join(dim_company, investments.investor_object_id == dim_company.object_id, 'left')
columns_to_drop = ['investor_object_id','object_id']
investments = investments.drop(*columns_to_drop)
RENAME_COLS = {
                "company_id": "investor_object_id"}
investments = investments.withColumnsRenamed(colsMap = RENAME_COLS)

In [80]:
investments=investments.join(fct_funding_rounds, investments.funding_round_id == fct_funding_rounds.funding_round_nk_id, 'left')
columns_to_drop = ['funding_round_nk_id','funding_round_id']
investments = investments.drop(*columns_to_drop)
RENAME_COLS = {
                "id": "funding_round_id"}
investments = investments.withColumnsRenamed(colsMap = RENAME_COLS)

In [81]:
investments.dtypes

[('investment_nk_id', 'int'),
 ('funded_object_id', 'string'),
 ('investor_object_id', 'string'),
 ('funding_round_id', 'string')]

In [82]:
investments.show()

+----------------+--------------------+--------------------+--------------------+
|investment_nk_id|    funded_object_id|  investor_object_id|    funding_round_id|
+----------------+--------------------+--------------------+--------------------+
|            1238|93c51854-4842-417...|390d7df2-b203-428...|760081e0-77cf-4e2...|
|            1591|3371cf11-7ca6-448...|10cd2dc7-b765-45f...|020ee3df-3afd-456...|
|            1645|2ed049ff-bbbb-45e...|57e5de6e-fb6b-45f...|81ef7fc8-6112-4a0...|
|             471|a9474f71-6802-4b5...|9ec1db6e-d756-481...|0d4ac18c-1eb9-48d...|
|             833|8be840ee-4d3b-425...|9cd43652-63d4-403...|3dd6e321-9c25-465...|
|            3794|dc379f91-aba2-405...|75846f37-f936-422...|65511a0d-ca68-4fd...|
|            1088|7278e7a2-6211-450...|03c88de2-9085-49f...|5c65cd00-bd0d-489...|
|            1342|7fc5632f-8c45-4fd...|20192e5e-fa88-4a6...|437d79ad-9bc3-425...|
|            1829|20546f88-cd93-428...|6ab4b0a1-7bd8-4de...|bc34ae5f-704a-447...|
|            195

In [83]:
load_dwh(investments,"fct_investments")

===== Get only new data =====
===== Start Extracting fct_investments data =====
===== Success Extracting fct_investments data =====
===== Already got only New Data  =====
===== Start Loading fct_investments new data =====
===== Success Loading fct_investments new data =====


In [2]:
trans=Transform()

===== Start Extracting people data =====
===== Success Extracting people data =====
===== Start Extracting relationships data =====
===== Success Extracting relationships data =====
===== Start Extracting acquisition data =====
===== Success Extracting acquisition data =====
===== Start Extracting company data =====
===== Success Extracting company data =====
===== Start Extracting funds data =====
===== Success Extracting funds data =====
===== Start Extracting investments data =====
===== Success Extracting investments data =====
===== Start Extracting ipos data =====
===== Success Extracting ipos data =====
===== Start Extracting funding_rounds data =====
===== Success Extracting funding_rounds data =====


In [3]:
dim_term_code=trans.dim_term_code()

In [4]:
dim_stock_symbol=trans.dim_stock_symbol()
dim_company=trans.dim_company()
dim_people=trans.dim_people()